In [ ]:
###### PARAM FOR LAST GOOD DATE BELOW TO BE SET ######
###### PARAM FOR LAST COMPLETE MONTH DOWNSTREAM TO BET MANUALLY SET! ####

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import requests

sns.set()

In [ ]:
#### the pymc models are weird - most times the don't sample first time ####


In [ ]:
##### excess calculation example #####

df_foo = pd.DataFrame({'pop' : [100,1000],'dead' : [1,20]},index=['Y1','Y2'])
excess_foo = df_foo.loc['Y2','dead'] - df_foo.loc['Y1','dead']
df_foo['mortality'] = df_foo['dead'] / df_foo['pop']
diff_mortality = df_foo.loc['Y2','mortality'] - df_foo.loc['Y1','mortality']
print (df_foo)
print ('abs_exess',excess_foo)
print ('diff mortality',diff_mortality)
print ('scaled excess', diff_mortality * df_foo.loc['Y2','pop'])


In [ ]:
def parse_scb_prel(YTD=True):
    
    def monthname2number(m):
        if m == 'januari' : return 1
        if m == 'februari' : return 2
        if m == 'mars' : return 3
        if m == 'april' : return 4
        if m == 'maj' : return 5
        if m == 'juni' : return 6
        if m == 'juli' : return 7
        if m == 'augusti' : return 8
        if m == 'september' : return 9
        if m == 'oktober' : return 10
        if m == 'november' : return 11
        if m == 'december' : return 12

    df = pd.read_csv('scb-dead_per_day.csv',sep=';',usecols=[1,2,3,4,5,6,7,10,11],thousands=',')
    
    
    df['month'] = df['Månad'].apply(monthname2number)
    df[['Månad','month']] = df[['Månad','month']].replace(np.nan,0)
    df['month'] = df['month'].astype(int)
    
    
    df = df.replace(np.nan,0)

    
    df[['2015','2016','2017','2018','2019','2020','2015-2019']] = df[['2015','2016','2017','2018','2019','2020','2015-2019']].astype(int)

    unknown_death_day = df.iloc[-1,:7]
    
    df.drop(366,inplace=True)
    
    if YTD:
        df = df.loc[df['2020'] != 0]
        
    df = df.drop('Månad',axis=1)
    
    known_death_day = df.iloc[:,:7].sum()
    
    year_fraction = len(df) / 366
    
    unknown_death_day[['2015','2016','2017','2018','2019','2015-2019']] = unknown_death_day[
    ['2015','2016','2017','2018','2019','2015-2019']] * year_fraction
    
    dead_pct_per_day = df[['2015','2016','2017','2018','2019','2020','2015-2019']] / known_death_day
    
    all_deaths = df[['2015','2016','2017','2018','2019','2020','2015-2019']] + unknown_death_day * dead_pct_per_day
    
    if YTD:
        index = pd.date_range('2020-01-01',periods=len(all_deaths))
    else:
        index = pd.date_range('2020-01-01','2020-12-31')
        
    all_deaths.index = index
    
    return all_deaths
    
    

In [ ]:
df = parse_scb_prel()


In [ ]:
df

In [ ]:
df['2015-2018_avg'] = df.loc[:,'2015' : '2018'].mean(axis=1)

In [ ]:
#### PARAM ##### 
last_good_date = '2020-11-15'
df = df.loc[:last_good_date].copy()
df

In [ ]:
df.sum()

In [ ]:
df['excess'] = df['2020'] - df['2015-2018_avg']
df.drop('2015-2019',axis=1,inplace=True)
df

In [ ]:
df.sum()

In [ ]:
pop = pd.read_pickle('pop_2015_2020.pkl')
pop.loc['excess'] = pop.loc['2020']
pop.loc['2015-2018_avg'] = pop.loc['2015' : '2018'].mean()

### setting the baseline pop to 2020 yields correct erroneous excess !!! #
#pop.loc['2015-2018_avg'] = pop.loc['2020']
###

pop

In [ ]:
## for excess to be correct, the diff must be obtained from dividing the abs diff with the pop ! ##
df_per_m = df / (pop / 1e6 )

# taking the diff after having made fractions yields wrong result! b/c populations are different!

# that is: df_per_m['excess'] which gives correct result is divided by population 2020, 
# while df_per_m['wrong_excess'] is calculated by diff 2020 - baseline, where the deaths are 
# already divided by population 2015-2018_avg, a smaller population which results in larger abs numbers. 

### BOTTOM LINE : dont add/subtract fractions where the denominator is different! ###

df_per_m['wrong_excess'] = df_per_m['2020'] - df_per_m['2015-2018_avg']

df_per_m

In [ ]:
# if you subtract fractions youll get wrong result! #
per_m_diff = df_per_m['2020'] - df_per_m['2015-2018_avg']
per_m_diff.sum()

In [ ]:
df_per_m.sum()

In [ ]:
df_per_m.cumsum().plot(y=['excess','wrong_excess'])

In [ ]:
df['excess'].cumsum().plot()

In [ ]:
df.sum() / pop

In [ ]:
(df['excess'] / (pop.loc['2020'] / 1e6)).cumsum()

In [ ]:
df.sum().loc['2020'] - df.sum().loc['2015-2018_avg']

In [ ]:
df.sum()

In [ ]:
pop_diff = pop.loc['2020'] - pop.loc['2015-2018_avg']
pop_diff

In [ ]:
dead_diff = df.sum().loc['2020'] - df.sum().loc['2015-2018_avg']
dead_diff

In [ ]:
dead_diff / (pop.loc['2020'] / 1e6)

In [ ]:
df_per_m.sum()

In [ ]:
dead_per_m_2020 = df_per_m.sum().loc['2020']
dead_per_m_15_18 = df_per_m.sum().loc['2015-2018_avg']

print (dead_per_m_15_18)
print (dead_per_m_2020)

In [ ]:
# diff in cum mortality expressed in millions
dead_per_m_2020 - dead_per_m_15_18

In [ ]:
# diff in mortality
mort_diff = ((df.sum() / pop).loc['2020'] - (df.sum() / pop).loc['2015-2018_avg']) 
mort_diff

In [ ]:
mortality = df_per_m.sum() / 1e6
mortality


In [ ]:
dead_2020 = mortality.loc['2020'] * pop.loc['2020']
dead_15_18 = mortality.loc['2015-2018_avg'] * pop.loc['2015-2018_avg']

print (dead_2020)
print (dead_15_18)

print (dead_2020 - dead_15_18)

In [ ]:
pop

In [ ]:
abs_totals = df.sum()
abs_totals

In [ ]:
mortality = abs_totals / pop
mortality

In [ ]:
mortality.loc['excess']

In [ ]:
mort_diff * pop.loc['2020']

In [ ]:
calc_deaths = mortality * pop
calc_deaths

In [ ]:
calc_diff = calc_deaths.loc['2020'] - calc_deaths.loc['2015-2018_avg']
calc_diff

In [ ]:
mortality

In [ ]:
pop_adj_deaths = mortality * pop.loc['2020']

pop_adj_deaths.index = pd.Categorical(pop_adj_deaths.index,
                                      ['2015','2016','2017','2018','2019','2015-2018_avg','2020','excess'])

pop_adj_deaths.sort_index(inplace=True)
pop_adj_deaths

In [ ]:
colors = ['r'] * 5
colors.extend(['crimson','crimson'])


title = 'SWEDEN population adjusted [std year 2020] deaths 2015-2020 YTD {}'.format(df_per_m.index[-1].date())
pop_adj_deaths[:-1].plot(kind='bar',figsize=(18,12),title=title,color=colors)
_= plt.yticks(np.arange(0,90000,5000))
plt.ylabel('pop adjusted deaths [std year 2020]')
plt.savefig('swe_pop_adj_deaths_ytd_15_20.jpg',format='jpg')


In [ ]:
df.sum()

In [ ]:
from pandas.tseries.offsets import MonthEnd
month_ends = df_per_m.index + MonthEnd(0)
month_ends

In [ ]:
#### historical monthly deaths

url = 'http://api.scb.se/OV0104/v1/doris/sv/ssd/START/BE/BE0101/BE0101G/ManadFoddDod'

data = {
  "query": [
    {
      "code": "Kon",
      "selection": {
        "filter": "item",
        "values": [
          "1",
          "2"
        ]
      }
    },
    {
      "code": "Manad",
      "selection": {
        "filter": "item",
        "values": [
          "01",
          "02",
          "03",
          "04",
          "05",
          "06",
          "07",
          "08",
          "09",
          "10",
          "11",
          "12",
          "90"
        ]
      }
    },
    {
      "code": "ContentsCode",
      "selection": {
        "filter": "item",
        "values": [
          "000000NF"
        ]
      }
    }
  ],
  "response": {
    "format": "json"
  }
}

In [ ]:
r = requests.post(url,json=data)

In [ ]:
r.status_code

In [ ]:
json = r.json()
json['data'][0]

In [ ]:
data_list = []

nr_records = (len(json['data']))

for rec in range(nr_records):

    data_list.append ((json['data'][rec]['key'][0],json['data'][rec]['key'][1],json['data'][rec]['key'][2],
                        json['data'][rec]['values'][0]))



In [ ]:
hist_deaths = pd.DataFrame(data_list,columns=['gender','month','year','deaths'])
hist_deaths['deaths'] = hist_deaths['deaths'].astype(int)
hist_deaths.head()

In [ ]:
hist_deaths = hist_deaths.groupby(['year','month']).sum()
hist_deaths

In [ ]:
yearly_sums = hist_deaths.groupby('year').sum()
yearly_sums

In [ ]:
unknown_month_mask = hist_deaths.index.get_level_values('month') == '90'
unknown_month_mask

In [ ]:
unknown_death_months = hist_deaths.loc[unknown_month_mask]['deaths']
unknown_death_months = unknown_death_months.reset_index().set_index('year')
unknown_death_months

In [ ]:
hist_deaths = (hist_deaths.loc[~unknown_month_mask]).copy()
hist_deaths

In [ ]:
# wow! 
hist_deaths['ratio_of_year'] = hist_deaths / hist_deaths.groupby('year').sum()
hist_deaths

In [ ]:
hist_deaths.groupby('year').sum()

In [ ]:
hist_deaths


In [ ]:
hist_deaths = hist_deaths.reset_index().set_index('year')
hist_deaths

In [ ]:
hist_deaths['corr_deaths'] = unknown_death_months['deaths'] * hist_deaths['ratio_of_year'] + hist_deaths['deaths']
hist_deaths['delta'] = hist_deaths['corr_deaths'] - hist_deaths['deaths']
hist_deaths

In [ ]:
hist_deaths.groupby(hist_deaths.index).sum()

In [ ]:
yearly_sums

In [ ]:
hist_deaths

In [ ]:
dates = pd.to_datetime(hist_deaths.index + '-' + hist_deaths['month'],infer_datetime_format=True)
dates

In [ ]:
hist_deaths.index = dates
hist_deaths

In [ ]:
hist_deaths_ytd = hist_deaths.copy()
hist_deaths_ytd = hist_deaths_ytd['corr_deaths']
hist_deaths_ytd

In [ ]:
(hist_deaths_ytd[hist_deaths_ytd.index.get_level_values(0) == 2019]).sum()

In [ ]:
df = df.astype(float)
df.info()

In [ ]:
df_monthly = df.resample('M').sum()
df_monthly.sum()

In [ ]:
hist_deaths_ytd = pd.concat([hist_deaths_ytd,df_monthly['2020']])
hist_deaths_ytd

In [ ]:
hist_deaths_ytd.tail(50)

In [ ]:
hist_deaths_ytd.plot(figsize=(18,12))

In [ ]:
plt.figure(figsize=(18,12))
plt.title('SWEDEN monthly abs deaths 1851 - 2020 YTD {}'.format(hist_deaths_ytd.index[-2].date()))
plt.plot(hist_deaths_ytd[:-1])
plt.ylabel('number of deaths per month')
plt.savefig('swe_historical_abs_deaths.jpg',format='jpg')

In [ ]:
## population 1860 - 2020

pop_url = 'http://api.scb.se/OV0104/v1/doris/sv/ssd/START/BE/BE0101/BE0101A/BefolkningR1860'

pop_data = {
  "query": [
    {
      "code": "Alder",
      "selection": {
        "filter": "vs:Ålder1årA",
        "values": [
          "0",
          "1",
          "2",
          "3",
          "4",
          "5",
          "6",
          "7",
          "8",
          "9",
          "10",
          "11",
          "12",
          "13",
          "14",
          "15",
          "16",
          "17",
          "18",
          "19",
          "20",
          "21",
          "22",
          "23",
          "24",
          "25",
          "26",
          "27",
          "28",
          "29",
          "30",
          "31",
          "32",
          "33",
          "34",
          "35",
          "36",
          "37",
          "38",
          "39",
          "40",
          "41",
          "42",
          "43",
          "44",
          "45",
          "46",
          "47",
          "48",
          "49",
          "50",
          "51",
          "52",
          "53",
          "54",
          "55",
          "56",
          "57",
          "58",
          "59",
          "60",
          "61",
          "62",
          "63",
          "64",
          "65",
          "66",
          "67",
          "68",
          "69",
          "70",
          "71",
          "72",
          "73",
          "74",
          "75",
          "76",
          "77",
          "78",
          "79",
          "80",
          "81",
          "82",
          "83",
          "84",
          "85",
          "86",
          "87",
          "88",
          "89",
          "90",
          "91",
          "92",
          "93",
          "94",
          "95",
          "96",
          "97",
          "98",
          "99",
          "100+"
        ]
      }
    },
    {
      "code": "Kon",
      "selection": {
        "filter": "item",
        "values": [
          "1",
          "2"
        ]
      }
    }
  ],
  "response": {
    "format": "json"
  }
}

In [ ]:
r_pop = requests.post(pop_url, json=pop_data)

In [ ]:
r_pop.status_code

In [ ]:
pop_json = r_pop.json()
pop_json['data'][0]

In [ ]:
pop_list = []

nr_records = (len(pop_json['data']))

for rec in range(nr_records):

    pop_list.append ((pop_json['data'][rec]['key'][0],pop_json['data'][rec]['key'][1],
                       pop_json['data'][rec]['key'][2],
                        pop_json['data'][rec]['values'][0]))



In [ ]:
l = list(range(0,100))

str_l = [str(i) for i in l]

str_l.append('100+')
str_l

In [ ]:
pop_df = pd.DataFrame(pop_list,columns=['age','gender','year','pop'])
pop_df['pop'] = pop_df['pop'].astype(int)
pop_df['age'] = pd.Categorical(pop_df['age'],str_l)


In [ ]:
pop_df = pop_df.sort_values(['year','age'])
pop_df

In [ ]:
pop_df = pop_df.groupby(['year','age']).sum()
pop_df

In [ ]:
pop_df.loc['2019'].sum()

In [ ]:
total_pops = pop_df.groupby('year').sum()
total_pops.index = pd.to_datetime(total_pops.index).year

In [ ]:
total_pops

In [ ]:
hist_deaths_ytd.resample('Y').sum()

In [ ]:
yearly_abs_deaths = hist_deaths_ytd.resample('Y').sum()

yearly_abs_deaths.index = yearly_abs_deaths.index.year

In [ ]:
yearly_abs_deaths

In [ ]:
# includes partial 2020 deaths ! #

title='SWEDEN population 1861 - 2019'
#ax = yearly_abs_deaths[:-1].plot(label='abs_deaths',style='ro--',figsize=(18,12),title=title)
#ax.legend(loc='upper left')
#ax.set_ylabel('abs deaths')
#ax = plt.gca()
#ax2 = plt.twinx()
total_pops.plot(style='bo--',figsize=(18,12),title=title)
plt.ylabel('population')
plt.xlabel('year')
plt.legend(loc='upper left')
#ax2.legend(loc='upper right')
#ax2.set_ylabel('population')

plt.savefig('awe_pop_1861_2019.jpg',format='jpg')

In [ ]:
hist_deaths_ytd

In [ ]:
total_pops.loc['2020','pop'] = np.nan
total_pops = total_pops.shift()
total_pops

In [ ]:
foo = hist_deaths_ytd.groupby([hist_deaths_ytd.index.year]).sum()
foo.index.name='year'
foo

In [ ]:
hist_deaths_ytd = pd.DataFrame(hist_deaths_ytd)
hist_deaths_ytd['year'] = hist_deaths_ytd.index.year
hist_deaths_ytd.columns=['dead','year']

In [ ]:
hist_deaths_ytd

In [ ]:
total_pops.index = pd.to_datetime(total_pops.index,format='%Y')
total_pops['y'] = total_pops.index.year
total_pops

In [ ]:
total_pops.reset_index()

In [ ]:
hist_deaths_ytd = hist_deaths_ytd.reset_index()
hist_deaths_ytd

In [ ]:
hist_deaths_ytd = hist_deaths_ytd.set_index('year')
hist_deaths_ytd

In [ ]:
total_pops = total_pops.set_index('y')
total_pops

In [ ]:
mortality_per_month = hist_deaths_ytd['dead'] / total_pops['pop']
mortality_per_month = pd.DataFrame(mortality_per_month,columns=['mortality'])
mortality_per_month

In [ ]:
mortality_per_month.index = hist_deaths_ytd['index']
mortality_per_month['month'] = mortality_per_month.index.month
mortality_per_month

In [ ]:
mortality_per_month = mortality_per_month.dropna()

In [ ]:
mortality_per_month

In [ ]:
### param ### 
last_complete_month = 10
###

full_mortality_per_month = mortality_per_month.copy()

mortality_per_month = mortality_per_month.loc[mortality_per_month['month'] <= last_complete_month]


In [ ]:
yearly_mortality = mortality_per_month.groupby(mortality_per_month.index.year).sum()
yearly_mortality

In [ ]:
title = 'SWEDEN monthly mortality 1861 - 2020, for period Jan 1:st - {} last'.format(
mortality_per_month.index[-1].month_name())
mortality_per_month.plot(y='mortality',figsize=(18,12),title=title)
plt.ylabel('YTD monthly mortality')
plt.xlabel('year')

In [ ]:
mortality_per_month.index[-1].date()

In [ ]:
yearly_mortality.plot(y='mortality',figsize=(18,12),style='o--',
                      title='SWEDEN mortality YTD 1861 - 2020, for period Jan 1st - {} last'.format(
mortality_per_month.index[-1].month_name()))

plt.ylabel('YTD mortality')
plt.xlabel('year')

plt.savefig('swe_historical_mortality_YTD_1861_2020.jpg',format='jpg')

In [ ]:
(mortality_per_month.loc['2020-01-01' :,'mortality' ]).plot(style='o--')
mortality_per_month

In [ ]:
year_month_mortality = mortality_per_month.groupby([mortality_per_month.index.year,
                                                    mortality_per_month.index.month]).mean()

year_month_mortality = year_month_mortality.droplevel(level=1)
year_month_mortality

In [ ]:
avg_mortality_15_18 = year_month_mortality.loc['2015' : '2018'].groupby('month').mean()
avg_mortality_15_18

In [ ]:
monthly_mort_2020 = mortality_per_month.loc['2020-01-01' :,'mortality']
monthly_mort_2020

In [ ]:
title = 'SWEDEN 2020 monthly mortality Jan 1:st - {} last cmp baseline 2015-2018'.format(
    mortality_per_month.index[-1].month_name())

ax = avg_mortality_15_18.reset_index().plot(y='mortality',title=title,
                                            style='o--',label='avg 15-18',figsize=(18,12))

(mortality_per_month.loc['2020-01-01' :,'mortality' ]).reset_index().plot(style='o--',y='mortality',
                                                                          label='2020',ax=ax)

plt.xticks(range(len(avg_mortality_15_18)),range(1,len(avg_mortality_15_18) + 1))
plt.xlabel('month')
plt.ylabel('monthly mortality')
plt.legend(loc='upper left')

ax2 = plt.twinx()

ax2.plot(avg_mortality_15_18['mortality'].values.cumsum(),'x--',label='avg 15-18 cumulative mortality YTD')
ax2.plot(monthly_mort_2020.values.cumsum(),'x--',label='2020 cumulative mortality YTD')

ax2.set_ylabel('cumulative mortality YTD')
ax2.legend(loc='upper center')
plt.savefig('swe_monthly_mortality_vs_baseline.jpg',format='jpg')

In [ ]:
death_url = 'http://api.scb.se/OV0104/v1/doris/sv/ssd/START/BE/BE0101/BE0101I/DodaFodelsearK'

death_data = {
  "query": [
    {
      "code": "Region",
      "selection": {
        "filter": "vs:RegionRiket99",
        "values": [
          "00"
        ]
      }
    },
    {
      "code": "Alder",
      "selection": {
        "filter": "vs:Ålder1årA",
        "values": [
          "0",
          "1",
          "2",
          "3",
          "4",
          "5",
          "6",
          "7",
          "8",
          "9",
          "10",
          "11",
          "12",
          "13",
          "14",
          "15",
          "16",
          "17",
          "18",
          "19",
          "20",
          "21",
          "22",
          "23",
          "24",
          "25",
          "26",
          "27",
          "28",
          "29",
          "30",
          "31",
          "32",
          "33",
          "34",
          "35",
          "36",
          "37",
          "38",
          "39",
          "40",
          "41",
          "42",
          "43",
          "44",
          "45",
          "46",
          "47",
          "48",
          "49",
          "50",
          "51",
          "52",
          "53",
          "54",
          "55",
          "56",
          "57",
          "58",
          "59",
          "60",
          "61",
          "62",
          "63",
          "64",
          "65",
          "66",
          "67",
          "68",
          "69",
          "70",
          "71",
          "72",
          "73",
          "74",
          "75",
          "76",
          "77",
          "78",
          "79",
          "80",
          "81",
          "82",
          "83",
          "84",
          "85",
          "86",
          "87",
          "88",
          "89",
          "90",
          "91",
          "92",
          "93",
          "94",
          "95",
          "96",
          "97",
          "98",
          "99",
          "100+"
        ]
      }
    },
    {
      "code": "Kon",
      "selection": {
        "filter": "item",
        "values": [
          "1",
          "2"
        ]
      }
    }
  ],
  "response": {
    "format": "json"
  }
}

In [ ]:
r_death_age = requests.post(death_url,json=death_data)
r_death_age.status_code

In [ ]:
death_age_json = r_death_age.json()
death_age_json['data'][0]

In [ ]:
death_age_list = []

nr_records = (len(death_age_json['data']))

for rec in range(nr_records):

    death_age_list.append ((death_age_json['data'][rec]['key'][0],death_age_json['data'][rec]['key'][1],
                       death_age_json['data'][rec]['key'][2],death_age_json['data'][rec]['key'][3],
                        death_age_json['data'][rec]['values'][0]))



In [ ]:
death_age_df = pd.DataFrame(death_age_list,columns=['riket','age','gender','year','deaths'])
death_age_df['deaths'] = death_age_df['deaths'].astype(int)
death_age_df

In [ ]:
death_age_df['age'] = pd.Categorical(death_age_df['age'],str_l)

death_age_df = death_age_df.groupby(['year','age']).sum()

death_age_df

In [ ]:
pop_df

In [ ]:
age_mortality = death_age_df['deaths'] /pop_df['pop'] 
age_mortality.dropna(inplace=True)

In [ ]:
age_mortality = pd.DataFrame(age_mortality,columns=['mortality'])
age_mortality = age_mortality.reset_index().set_index('year')
age_mortality

In [ ]:
pop_df_mask = pop_df.index.get_level_values(0) == '2019'
age_std_pop = pop_df[pop_df_mask]
age_std_pop = age_std_pop.reset_index().set_index('year')
age_std_pop

In [ ]:
nr_unique_years = len(age_mortality.index.unique())

age_mortality['std_pop'] = np.tile(age_std_pop['pop'],nr_unique_years)
age_mortality['age_adj_deaths'] = age_mortality['mortality'] * age_mortality['std_pop']
age_mortality

In [ ]:
age_adj_deaths = age_mortality.groupby('year').sum()
age_adj_deaths['age_pop_adj_mortality'] = age_adj_deaths['age_adj_deaths'] / age_std_pop['pop'].sum()
age_adj_deaths

In [ ]:
yearly_mortality

In [ ]:

age_adj_deaths.plot(kind='bar',y='age_pop_adj_mortality',figsize=(18,12))
_= plt.yticks(np.arange(0,0.02,0.001))

In [ ]:
yearly_mortality.loc[1968:2019]

In [ ]:
pop_df

In [ ]:
death_age_df.reset_index(inplace=True)

In [ ]:
def str_2_int(s):
    if s[-1] == '+':
        return int(s[:-1])
    return int(s)

death_age_df['age_num'] = (death_age_df['age'].apply(str_2_int)).astype(int)

In [ ]:
death_age_df

In [ ]:
death_age_df['grp'] = pd.cut(death_age_df.reset_index()['age_num'],[-1,64,79,89,1000])
death_age_df

In [ ]:
grp_death = death_age_df.groupby(['year','grp']).sum()
grp_death

In [ ]:
pop_df.reset_index(inplace=True)
pop_df['age_num'] = (pop_df['age'].apply(str_2_int)).astype(int)

In [ ]:
pop_df

In [ ]:
pop_df['grp'] = pd.cut(pop_df['age_num'],[-1,64,79,89,1000])
pop_df

In [ ]:
grp_pop = pop_df.groupby(['year','grp']).sum()
grp_pop

In [ ]:
age_pop_dead = pd.concat([grp_pop['pop'],grp_death['deaths']],axis=1)
age_pop_dead

In [ ]:
age_pop_dead.dropna(inplace=True)

In [ ]:
age_pop_dead

In [ ]:
(age_pop_dead[age_pop_dead.index.get_level_values(0) == '2019']).sum()

In [ ]:
age_pop_dead['mortality'] = age_pop_dead['deaths'] / age_pop_dead['pop']
age_pop_dead

In [ ]:
# 2020 age deaths

### can't do age adj deaths incl 2020 YTD b/c cant find historical death data with both month and age ###

df_age_2020 = pd.read_csv('scb_dead_per_age.csv',sep=';',header=None,index_col=0,
                          usecols=[0,11,12,13,14,15,16,16,17,18],thousands=',')

df_age_2020[64] = df_age_2020[11] + df_age_2020[15]
df_age_2020[79] = df_age_2020[12] + df_age_2020[16]
df_age_2020[89] = df_age_2020[13] + df_age_2020[17]
df_age_2020[90] = df_age_2020[14] + df_age_2020[18]

df_age_2020 = df_age_2020.drop([11,12,13,14,15,16,17,18],axis=1)
df_age_2020

In [ ]:
unknown_days = df_age_2020.iloc[-1,:]
unknown_days

In [ ]:
df_age_2020 = (df_age_2020[(df_age_2020 == 0) == False]).dropna()[:-1]
df_age_2020 = df_age_2020[:-7]
df_age_2020

In [ ]:
add_ons = df_age_2020 / df_age_2020.sum() * unknown_days
add_ons

In [ ]:
df_age_2020 = df_age_2020 + add_ons
df_age_2020

In [ ]:
grps_2018 = grp_pop[grp_pop.index.get_level_values(0) == '2018']
grps_2019 = grp_pop[grp_pop.index.get_level_values(0) == '2019']

grp_growth = grps_2019['pop'].values / grps_2018['pop'].values
grp_growth

In [ ]:
grps_2020_extrapolated = grp_pop[grp_pop.index.get_level_values(0) == '2019']['pop'] * grp_growth
grps_2020_extrapolated = grps_2020_extrapolated.reset_index()
grps_2020_extrapolated['year'] = '2020'
grps_2020_extrapolated

In [ ]:
grp_pop = grp_pop.reset_index()
grp_pop

In [ ]:
grp_pop = grp_pop.append(grps_2020_extrapolated)
grp_pop

In [ ]:
grps_2020_extrapolated['grp']

In [ ]:
deaths_2020_YTD = df_age_2020.sum()
deaths_2020_YTD = pd.DataFrame(deaths_2020_YTD)
deaths_2020_YTD['year'] = ['2020'] * 4
deaths_2020_YTD.rename(columns={0:'deaths'},inplace=True)
deaths_2020_YTD['grp'] = grps_2020_extrapolated['grp'].values
deaths_2020_YTD

In [ ]:
grp_death = grp_death.reset_index().append(deaths_2020_YTD)
grp_death

In [ ]:
grp_pop = grp_pop.set_index(['year','grp'])
grp_death = grp_death.set_index(['year','grp'])

grp_pop

In [ ]:
grp_death

In [ ]:
# full year mortality until 2019, 2020 YTD

grp_mortality_YTD = pd.DataFrame((grp_death['deaths'] / grp_pop['pop']).dropna(),columns=['mortality'])
grp_mortality_YTD

In [ ]:
grp_std_pop = grp_pop[grp_pop.index.get_level_values(0) == '2019']
grp_std_pop

In [ ]:
nr_years = len(grp_mortality_YTD.reset_index()['year'].unique())

In [ ]:
np.tile(grp_std_pop['pop'].values,nr_years)
grp_mortality_YTD['std_pop'] = np.tile(grp_std_pop['pop'].values,nr_years)
grp_mortality_YTD['age_adj_deaths'] = grp_mortality_YTD['mortality'] * grp_mortality_YTD['std_pop']
grp_mortality_YTD

In [ ]:
age_adj_deaths_inc_2020 = grp_mortality_YTD.groupby('year').sum()
####
age_adj_deaths_inc_2020.drop('mortality',inplace=True,axis=1)
####
age_adj_deaths_inc_2020

In [ ]:
nr_years

In [ ]:
colors = ['b'] * (nr_years -1) 
colors.extend(['orange'])
print (colors)

title = "SWEDEN age adj deaths [std-year 2019] 1968-2020\n(2020 data only until {})".format(df_age_2020.index[-1])

_= age_adj_deaths_inc_2020.plot(kind='bar',y='age_adj_deaths',figsize=(18,12),title=title,color=colors)

In [ ]:
# 2020 months reg at month end, rest month start
hist_deaths_ytd['rank'] = hist_deaths_ytd['dead'].rank(ascending=True)
hist_deaths_ytd

In [ ]:
hist_deaths_ytd.loc[hist_deaths_ytd['index'] == '2019-12-01']

In [ ]:
hist_deaths_ytd.sort_values('rank')[-50:]

In [ ]:
mortality_per_month['rank'] = mortality_per_month['mortality'].rank(ascending=True)
mortality_per_month

In [ ]:
mortality_per_month.sort_values('rank',ascending=False)[:50]

In [ ]:
mortality_per_month.sort_values('rank')[:50]

In [ ]:
mortality_per_month.sort_values('rank',ascending=True)[:50]

In [ ]:
mortality_per_month.index = mortality_per_month.index.date


In [ ]:
mortality_per_month[pd.DatetimeIndex(mortality_per_month.index).year == 2020]

In [ ]:
top_20 = mortality_per_month.sort_values('rank',ascending=True)[:20]
top_20

In [ ]:
top_20_2020 = pd.DatetimeIndex(top_20.index).year == 2020
top_20_2020

In [ ]:

colors = ['b' if not top_20_2020[i] else 'r' for i in range(len(top_20_2020))]
colors 

In [ ]:
title = 'SWEDEN Top-20 lowest mortality months, 1861 - 2020 YTD {}'.format(mortality_per_month.index[-1])

mortality_per_month.sort_values('rank',ascending=True)[:20].plot(figsize=(18,12),color=colors,
    kind='bar',y='mortality',title=title)

plt.ylabel('monthly mortality')

In [ ]:
mortality_per_month

In [ ]:
yearly_mortality

In [ ]:
age_adj_deaths

In [ ]:
yearly_mortality.info()

In [ ]:
age_adj_deaths.index = range(1968,2020)
age_adj_deaths.info()

In [ ]:
yearly_mortality['age_pop_adj_mortality'] = age_adj_deaths['age_pop_adj_mortality']
yearly_mortality

In [ ]:
title = 'SWEDEN mortality 1968 - 2019 : comparison mortality vs age adj mortality'
ax = yearly_mortality.dropna().plot(y=['mortality','age_pop_adj_mortality'],figsize=(18,12),
                              style=['o--','o--'])

In [ ]:
age_adj_deaths_inc_2020['age_pop_adj_mortality'] = age_adj_deaths_inc_2020['age_adj_deaths'] /\
age_adj_deaths_inc_2020['std_pop']

age_adj_deaths_inc_2020


In [ ]:
age_adj_deaths_inc_2020.index = range(1968,2021)

In [ ]:
title = 'SWEDEN mortality comparison 1968 - 2020\n2020 data YTD {}'.format(df_age_2020.index[-1])

ax = yearly_mortality.dropna().plot(y=['mortality','age_pop_adj_mortality'],figsize=(18,12),
                              style=['o--','o--'],title=title,
                                    label=['mortality','age_pop_adj_mortality yearly age grps'])

ax.plot(age_adj_deaths_inc_2020['age_pop_adj_mortality'],'o--',
        label='age_pop_ajd_mortality 4 age grps')
ax.legend(loc='upper right')

In [ ]:
colors = ['b'] * (nr_years -1) 
colors.extend(['orange'])
print (colors)

title = "SWEDEN age adj mortality [std-year 2019] \
1968-2020\n(2020 data YTD until {}, others full years)".format(df_age_2020.index[-1])

_= age_adj_deaths_inc_2020.plot(kind='bar',y='age_pop_adj_mortality',figsize=(18,12),title=title,color=colors)
plt.yticks(np.arange(0,0.018,0.001))
plt.savefig('swe_age_adj_mortality_1968_2020_YTD.jpg',format='jpg')

In [ ]:
year = hist_deaths_ytd['index']
month = year.dt.month

hist_deaths_ytd.rename(columns={'index' : 'month'},inplace=True)

In [ ]:
hist_deaths_ytd.set_index('month',inplace=True)
hist_deaths_ytd

In [ ]:
hist_cum_deaths = hist_deaths_ytd.groupby(hist_deaths_ytd.index.get_level_values(0).year).cumsum()
hist_cum_deaths

In [ ]:
deaths_year_month = hist_deaths_ytd.reset_index()

In [ ]:
deaths_year_month.rename(columns={'month': 'date'},inplace=True)
deaths_year_month

In [ ]:
deaths_year_month['year'] = deaths_year_month['date'].apply(lambda d : d.year)
deaths_year_month['month'] = deaths_year_month['date'].apply(lambda d : d.month)

In [ ]:
deaths_year_month

In [ ]:
deaths_year_month

In [ ]:
deaths_year_month = deaths_year_month.pivot(index='year',columns='month',values='dead')

In [ ]:
deaths_year_month

In [ ]:
sns.axes_style()

In [ ]:
start_year = 1900
sns.set()
sns.set_style({"grid.color": "lightgrey"})
title = 'SWEDEN abs deaths {} - 2020 YTD {}, stacked by month'.format(start_year,df_age_2020.index[-1])
deaths_year_month.loc[start_year:].plot(kind='bar',stacked=True,title=title,
                                        figsize=(18,12),edgecolor='none',fontsize=7)
ax = plt.gca()
for label in ax.xaxis.get_ticklabels()[1::2]:
        label.set_visible(False)


plt.savefig('swe_deaths_stacked_by_month_1860_2020_ytd.jpg',format='jpg')
_= plt.yticks(range(0,115000,5000))

In [ ]:
full_mortality_per_month['year'] = pd.DatetimeIndex(full_mortality_per_month.index).year
full_mortality_per_month

In [ ]:
full_mortality_year_month = full_mortality_per_month.pivot(index='year',columns='month',values='mortality')
full_mortality_year_month

In [ ]:
full_mortality_year_month.sum(axis=1)

In [ ]:
title = 'SWEDEN mortality {} - 2020 YTD {}, stacked by month'.format(start_year,df_age_2020.index[-1])

full_mortality_year_month.loc[start_year:].plot(kind='bar',
    stacked=True,figsize=(18,12),edgecolor='none',title=title,fontsize=7)

ax = plt.gca()
for label in ax.xaxis.get_ticklabels()[1::2]:
        label.set_visible(False)

_= plt.yticks(np.arange(0,0.019,0.001))
plt.legend(loc='upper center')
plt.savefig('swe_spot_the_pandemic.jpg',format='jpg')

In [ ]:
yearly_mortality = full_mortality_year_month.sum(axis=1)
yearly_mortality

In [ ]:
mortality_growth = yearly_mortality / yearly_mortality.shift()
mortality_growth = pd.DataFrame(mortality_growth,columns=['growth'])
mortality_growth['hue'] = np.random.randint(0,2,size=len(mortality_growth))
mortality_growth

In [ ]:
mean_growth = mortality_growth['growth'].mean()
mean_growth

In [ ]:
mortality_growth['growth_vs_avg'] = mortality_growth['growth'] / mean_growth
mortality_growth

In [ ]:
mortality_growth.plot(style='o--',y=['growth','growth_vs_avg'],figsize=(18,12))

In [ ]:
plt.figure(figsize=(18,12))
sns.violinplot(data=mortality_growth,y='growth',inner='box')
plt.title('SWEDEN Y2Y mortality growth factor distribution')
plt.xlabel('freq')
plt.ylabel('growth factor Y2Y')

In [ ]:


mortality_growth.plot(y='growth',style='o--')

In [ ]:
mortality_growth.dropna(inplace=True)

mortality_growth['growth'].plot(kind='hist')


In [ ]:
mortality_growth['growth'] 

In [ ]:
mortality_growth

In [ ]:
mortality_growth['growth'].plot()

In [ ]:
import pymc3 as pm
import arviz as az

model = pm.Model()
with model:
    x = range(len(mortality_growth))
    
    alpha = pm.Normal('alpha',mu=1,sd=1)
    beta = pm.Normal('beta',mu=0,sd=1)
    sigma = pm.Normal('sigma',1,3)
    
    reg = pm.Deterministic('reg', alpha + x * beta)
    
    obs = pm.Normal('obs',mu=reg,sd=sigma,observed=mortality_growth['growth'])
    
    trace = pm.sample(500,tune=500)
    

In [ ]:
with model:
    print (pm.summary(trace,hdi_prob=0.89))
    az.plot_trace(trace,var_names=['alpha','beta'])

In [ ]:
trace['reg']

In [ ]:
fig,ax = plt.subplots(figsize=(18,12))
with model:
    ppc = pm.sample_posterior_predictive(trace,samples=1000,model=model)
    
    az.plot_hpd(x,trace['reg'],hdi_prob=0.89,ax=ax)
    ax.plot(trace['alpha'].mean() + x * trace['beta'].mean(),color='k',ls='dashed')
    ax.plot(range(len(mortality_growth)),mortality_growth['growth'],'o')
    ax.set_xticks(range(len(mortality_growth)))
    ax.set_xticklabels(range(1862,2021),rotation=90)
    az.plot_hpd(x,ppc['obs'],hdi_prob=0.89,ax=ax)


In [ ]:
yearly_mortality *= 1000 # to get pymc to process the otherwise very small numbers
yearly_mortality

In [ ]:
yearly_mortality.plot()

In [ ]:
model2 = pm.Model()
with model2:
    

    
    x = range(len(yearly_mortality))
    
    alpha = pm.Normal('alpha',20,2)
    beta = pm.Normal('beta',0, 2)
    sigma = pm.Normal('sigma',10, 5)
    
    reg = pm.Deterministic('reg',alpha + x * beta)
    
    obs = pm.Normal('obs',reg, sigma,observed=yearly_mortality)
    
    trace2 = pm.sample(500,tune=500)
    

In [ ]:
with model2:
    print (pm.summary(trace2,hdi_prob=0.89))
    az.plot_trace(trace2,var_names=['alpha','beta','sigma'])

In [ ]:
fig,ax = plt.subplots(figsize=(18,12))

def numfmt(x, pos): # your custom formatter function: divide by 100.0
    s = '{}'.format(x / 1000.0)
    return s

import matplotlib.ticker as tkr     # has classes for tick-locating and -formatting
yfmt = tkr.FuncFormatter(numfmt)    # create your custom formatter function


with model2:
    
    ppc = pm.sample_posterior_predictive(trace2,samples=1000,model=model2)

    az.plot_hpd(x,trace2['reg'],hdi_prob=0.89,ax=ax)
    
    az.plot_hpd(x,ppc['obs'],hdi_prob=0.89,ax=ax)
    
    ax.plot(x,trace2['alpha'].mean() + x * trace2['beta'].mean(),color='k',ls='--')
    ax.plot(x,yearly_mortality,'o')
    ax.set_xticks(range(len(yearly_mortality)))
    ax.set_xticklabels(range(1861,2021),rotation=90)
    
    for label in ax.xaxis.get_ticklabels()[::2]:
        label.set_visible(False)
        
    ax.yaxis.set_major_formatter(yfmt)
        


In [ ]:
pop_df = pop_df.set_index(['year','grp'])
pop_df

In [ ]:
grouped_pop = pop_df.reset_index().groupby(['year','grp']).sum()
grouped_pop

In [ ]:
pop_pivot = (grouped_pop.reset_index()).pivot(index='year',columns='grp',values='pop')
pop_pivot

In [ ]:
pop_pivot.sum(axis=1)

In [ ]:
title = 'SWEDEN population stacked per age group, 1860-2019'
ax = pop_pivot.plot(kind='bar',stacked=True,figsize=(18,12),title=title)

for label in ax.xaxis.get_ticklabels()[::2]:
        label.set_visible(False)
        
plt.ylabel('population size')
plt.savefig('swe_abs_pop_age_grp_1860_2019.jpg',format='jpg')

In [ ]:
rel_pop_pivot = pop_pivot.div(pop_pivot.sum(axis=1),axis=0)
rel_pop_pivot

In [ ]:
title = 'SWEDEN age group proportion of total population, 1860-2019'


ax = rel_pop_pivot.plot(kind='bar',stacked=True,figsize=(18,12),title=title)

for label in ax.xaxis.get_ticklabels()[::2]:
        label.set_visible(False)

plt.ylabel('cumulative fraction of total population')
plt.savefig('swe_rel_pop_age_grp_1860_2019.jpg',format='jpg')        

In [ ]:
(pop_pivot / pop_pivot.shift()).plot(figsize=(18,12),style=['o--'] * 4)

In [ ]:
pop_growth_since_1860 = pop_pivot / pop_pivot.loc['1860']
pop_growth_since_1860

In [ ]:
pop_growth_since_1860.plot(figsize=(18,12))

In [ ]:
rel_pop_pivot

In [ ]:
rel_pop_growth_since_1860 = rel_pop_pivot / rel_pop_pivot.loc['1860']
rel_pop_growth_since_1860

In [ ]:
title = 'SWEDEN relative population age group growth, baseline 1860'
rel_pop_growth_since_1860.plot(figsize=(18,12),style=['o--'] * 4,title=title)
plt.ylabel('growth factor')
plt.savefig('swe_age_group_rel_growth.jpg',format='jpg')